# 词嵌入  
  
  自然语言是一套用来表达含义的复杂系统。在这套系统中，词是表意的基本单元。  
  顾名思义，词向量是用来表示词的向量。也可被认为是词的特征向量或表征。把词映射为实数域向量的技术也叫词嵌入。  
    
### 为何不采用one-hot向量  
  
* 用 one-hot 向量表示词   

  假设词典中不同词的数量N（词典中一共有N个词）。每个词可以和从0到N-1的连续数一一对应。这些与词对应的整数叫做词的索引i。为了得到词的one-hot表示，我们创建一个全0的长为N的向量，并把其第i位设为1。这样一来每个词都可以变为一个长度位N的one-hot向量，这样词就可以用来被神经网络使用。    
  
    
* 为什么不用one-hot  
  
  one-hot词向量构造起来容易，但通常不是一个好的选择。**一个主要的原因是：one-hot词向量无法准确的表达不同词之间的相似度。** 如我们常常使用余弦相似度来衡量不同词之间的相似度。对应向量$x,y$。它们的余弦相似度是这两个向量的夹角余弦值：$\frac{x^Ty}{|x||y|} \;\in R^d$  。而任何两个不相同词的one-hot向量的余弦相似度都为。所以one-hot体现不出来不同词之间的相似度。  
      

**词嵌入(word2vec)就是用来解决上述问题的。word2vec有两个模型：跳字模型（skip-gram）、连续词袋模型（continuous bag of word ,CBOW）**  

<br>   
  
### 词嵌入(word2vec)之跳字模型（skip-gram）  
  
  我们只是用one-hot(稀疏)向量去编码单词，并不能完全表达每一个单词的信息，也不能用来度量单词之间的相似性。  
    
  **现在的想法是用一个稠密的向量（复杂的向量）来表示一个单词，这个稠密的向量能完全表达单词的信息。（即这个向量是m维的，用m维特征来表示这个单词），做法用特定的损失函数把单词的one-hot向量编码到m维。**   
    
  $\color{red}{\text{通过特定的损失函数把单词的one-hot(稀疏)向量编码到稠密的向量，以期这个稠密向量能完全表达这个单词的全部信息。}}$  
  $\color{red}{\text{如果想让这个稠密的向量表达这个单词的不同信息。我们只需要调整不同信息对应的编码损失函数就行了。 }}$ 
    
**如何做（结构图）：**   
![](img/1.PNG)
![](img/2.PNG)
![](img/4.PNG)  
  
案例：  
  
 ["the", "man"," loves", "his", "son" ]  
   
编码后的one-hot向量：  

$$ \begin{bmatrix} the: &1 & 0 & 0 & 0 & 0 \\ man: & 0 & 1 & 0 & 0 & 0 \\loves: & 0 & 0 & 1 & 0 & 0\\his: &  0 & 0 & 0 & 1 & 0\\ son: & 0 & 0 & 0 & 0 & 1 \end{bmatrix} $$  
  
现在the的one-hot:[1,0,0,0,0] 并不能完全表示the这个单词的全部信息。我们想把这个one-hot向量进行编码让编码后的向量能完全表达这个单词的特征。   
  
$\color{red}{\text{现在确定这个单词要表达信息的编码损失函数。（有了对应的损失函数之后，就能把这个单词one-hot向量编码为其所代表的某个信息的稠密向量})}$   
  
  
**现在把one-hot编码到3维用来表示单词特征。编码的目标损失函数是核心词的周围词的概率**  
   
$$ \underset{one-hot}{ \begin{bmatrix} 1 & 0 & 0 & 0 & 0 \\ 0 & 1 & 0 & 0 & 0 \\ 0 & 0 & 1 & 0 & 0\\ 0 & 0 & 0 & 1 & 0\\  0 & 0 & 0 & 0 & 1 \end{bmatrix}} \times  \underset{linear-W}{\begin{bmatrix} 11 & 9 & 7 \\ 5 & 1 & 4 \\ 7 & 7 & 3 \\ 3 & 6 & 7 \\  8 & 0 & 10  \end{bmatrix}} = \underset{encoder}{ \begin{bmatrix} a & b & c \\ d & e & f \\ g & h & j \\ r & s & t\\  u & v & w  \end{bmatrix}} \rightarrow \underset{\text{极大化似然}}{ \frac{exp(encoder^T * encoder)}{\sum exp(encoder^T * encoder}}$$    
  
  
<br>  
  
**例子的完整描述：**
 
 跳字模型假设基于某个词来生成在它周围的词。例如 ["the", "man"," loves", "his", "son" ]，把loves作为中心词，把背景窗口设为2，跳字模型生成与中心词距离不超2的背景词the、man、his、son的条件概率。即  
 $$P("the", "man", "his", "son"|"loves" )$$  
   
现在假设给定中心词的情况下，背景词的生成相互独立，则:   
 $$P("the"|"loves"）P("man"|"loves"）P("his"|"loves"）P("son"|"loves"） $$   
 
 ![](img/3.PNG)  
   
 在跳字模型中，每个词可以被表示为两种向量，它在词典中索引为i,当它为中心词的时候，向量表示为$V_i$,当它为背景词的时候，向量表示为$u_i$。 则现在有一个中心词，其索引为c,则中心词为$w_c$,其特征表示向量为$V_c$。有一个背景词，其索引为o,则背景词为$w_o$,其特征表示向量为$U_o$ 。则根据中心词为$w_c$生成背景词为$w_o$的条件概率为：  
 $$P(w_0|w_c)=\frac{exp(u_o^T v_c)}{\sum_{i\in V }\;exp(u_i^T v_c)}$$  
   
 则设长度为T的文本序列，时间步t的词为$w^t$,背景窗口为m,则跳字模型的似然函数为（即给定一个中心词生成所有背景词的概率）：  
   
 $$\prod_{t=1}^T\prod_{-m\leq j \leq m ,j \neq 0 } P(w^{t+1}| w^t)$$  
   
<br>  

模型的训练：  
  
  用上式对中心词向量，和背景词向量$V_c，U_c$求梯度即可。  
    
<br>  

<br>  


### 词嵌入(word2vec)之连续词袋模型（continuous bag of word ,CBOW）  
  
  连续词袋模型与跳字模型类似。连续词袋模型是假设了某个中心词，根据中心词的前后背景词，来生成这个中心词的条件概率。  
    
 如["the", "man"," loves", "his", "son" ] 中假设loves为中心词，且背景窗口大小2，则连续跳字模型为：给定背景生成词["the", "man", "his", "son" ]生成中心词loves的条件概率  
   
  $$P(" loves"|"the", "man", "his", "son")$$  
    
 ![](img/5.PNG)     
   
同样把每个词表示为两种向量，它在词典中索引为i,当它为中心词的时候，向量表示为$V_i$,当它为背景词的时候，向量表示为$u_i$。因为背景词有多个所以把背景词向量取平均。  
  
  $$P(w_c|w_{o1},……，w_{o2m})=\frac{exp(\frac{1}{2m}u_c^T*(v_{o1}+……+v_{o2m}))}{\sum_{i \in V}\;exp(\frac{1}{2m}u_i^T*(v_{o1}+……+v_{o2m}))}$$  
    
假定一个长度为T的文本序列，设时间步t的词为$w^t$,背景窗口大小为m,连续词袋模型的似然函数是由背景词生成任一中心词的概率：  
  
 $$\prod_{t=1}^T P(w^{(t)}|w^{(t-m)},…,w^{(t-1)},w^{(t+1)}…，w^{(t+m)})$$  
   
**连续词袋模型的训练：**  
  
  模型的训练是一样的，用似然函数对各个词向量求梯度。  
  如：  
  ![](img/6.PNG)